# Blackstone (colab ready)

## Custom venv

blackstone 0.1.15 is fixed to spacy==2.1.8 so we have a custom venv for it.

`make venv_blackstone`  
`make jupyter_blackstone`  

https://pypi.org/project/blackstone/  
https://research.iclr.co.uk/blackstone  
https://github.com/ICLRandD/Blackstone  

In [ ]:
!pip install blackstone

In [ ]:
# NOTE: You may need to restart the kernel to be able to use these downloaded models if installed here
!pip install https://blackstone-model.s3-eu-west-1.amazonaws.com/en_blackstone_proto-0.0.1.tar.gz  # 243.3 MB
!python -m spacy download en_core_web_md  # v2.3.1 50.8 MB

In [ ]:
import datetime
import pandas as pd
import spacy
from spacy import displacy
from IPython.core.display import HTML

In [ ]:
nlps = {}
nlps["en_blackstone_proto"] = spacy.load("en_blackstone_proto")
nlps["en_core_web_md"] = spacy.load("en_core_web_md")

In [ ]:
start = datetime.datetime.now()

In [ ]:
text = """As we shall explain in more detail in examining the submission of the Secretary of State (see paras 77 and following), it is the Secretary of State’s case that nothing has been done by Parliament in the European Communities Act 1972 or any other statute to remove the prerogative power of the Crown, in the conduct of the international relations of the UK, to take steps to remove the UK from the EU by giving notice under article 50EU for the UK to withdraw from the EU Treaty and other relevant EU Treaties. The Secretary of State relies in particular on Attorney General v De Keyser’s Royal Hotel Ltd [1920] AC 508 and R v Secretary of State for Foreign and Commonwealth Affairs, Ex p Rees-Mogg [1994] QB 552; he contends that the Crown’s prerogative power to cause the UK to withdraw from the EU by giving notice under article 50EU could only have been removed by primary legislation using express words to that effect, alternatively by legislation which has that effect by necessary implication. The Secretary of State contends that neither the ECA 1972 nor any of the other Acts of Parliament referred to have abrogated this aspect of the Crown’s prerogative, either by express words or by necessary implication.
"""

In [ ]:
def apply_nlp(text, nlps, nlp_type, options_ent={}, render=True) -> pd.DataFrame:
    nlp = nlps[nlp_type]
    doc = nlp(text)
    
    if render:
        doc.user_data["title"] = f"{nlp_type}"
        #options_dep = {"compact": False, "bg": "#ffffff", "color": "grey", "font": "Source Sans Pro"}
        #displacy.render(doc, style="dep", options=options_dep)
        #displacy.render(list(doc.sents), style="dep", options=options_dep)
        displacy.render(doc, style="ent", jupyter=True, options=options_ent)
        
    ents = []
    for e in doc.ents:
        # print(ent.text, ent.label_)
        ents.append((e.label_, e.text, e.root, e.conjuncts, e.start_char, e.end_char, e.vector_norm))
    df = pd.DataFrame.from_records(ents)
    columns = ["label", "text", "root", "conjuncts", "start_char", "end_char", "vector_norm"]
    df.columns = columns
    df.sort_values(by="label", inplace=True)
    df = df.groupby(columns).size().reset_index(name='counts')
    return df

## Apply en_core_web_md to text

In [ ]:
df = apply_nlp(text, nlps, "en_core_web_md")
HTML(df.to_html())

## Apply en_blackstone_proto to text

In [ ]:
options = {"colors": {"CASENAME": "#EBDEF0", "CITATION": "#FADBD8", "INSTRUMENT": "#D5F5E3", "PROVISION": "#D6EAF8", "COURT": "#F7DC6F", "JUDGE": "#D6DBDF"}}
df = apply_nlp(text, nlps, "en_blackstone_proto", options)
HTML(df.to_html())

## Inspect ent object

In [ ]:
nlp = nlps["en_blackstone_proto"]
#nlp = nlps["en_core_web_md"]
doc = nlp(text)

In [ ]:
ent = doc.ents[3]
print("\n\nAll attrs:")
for name in [a for a in dir(ent) if not a.startswith('_')]:
    val = getattr(ent, name)
    print(f"{name} = {val}\n")

In [ ]:
for ent in doc.ents:
    print(f"Label *: {ent.label_}")
    print(f"Text  *: {ent.text}")
    print(f"Root   : {ent.root}")
    print(f"Conj   : {ent.conjuncts}")
    print(f"Start  : {ent.start_char}")
    print(f"End    : {ent.end_char}")
    print(f"VecNorm: {ent.vector_norm}")
    print("-----------------------------")